<a href="https://colab.research.google.com/github/sue-bin/sentiment-analysis-based-typography-generator/blob/main/goemotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/Colab\ Notebooks/NLP2

/content/gdrive/MyDrive/Colab Notebooks/NLP2


In [4]:
pip install transformers==2.11.0

In [5]:
pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [6]:
pip install attrdict==2.0.1

In [7]:
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
from pprint import pprint
import numpy as np
import pandas as pd
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

In [8]:
goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3
)

texts = [
    "Hey that's a thought! Maybe we need [NAME] to be the celebrity vaccine endorsement!",
    "it’s happened before?! love my hometown of beautiful new ken 😂😂",
    "I love you, brother.",
    "Troll, bro. They know they're saying stupid shit. The motherfucker does nothing but stink up libertarian subs talking shit",
]

pprint(goemotions(texts))

[{'labels': ['neutral'], 'scores': [0.9750906]},
 {'labels': ['curiosity', 'love'], 'scores': [0.9694574, 0.92274606]},
 {'labels': ['love'], 'scores': [0.993483]},
 {'labels': ['anger'], 'scores': [0.99225825]}]


In [9]:
def print_emotion(a):
  if a==2:
    pprint('anger')
  elif a==14:
    pprint('fear')
  elif a==17:
    pprint('happy')
  elif a==19:
    pprint('love')
  elif a==26:
    pprint('sadness')
  elif a==27:
    pprint('surprise')

여기서부터는 전처리 과정입니다! 전처리 끝난 모델을 저장해서 첨부하니까 여기부터 저 밑에까지는 실행하지 않으셔도 됩니다!!

In [10]:

df=pd.read_csv('Emotion_final.csv')
temp=df.head(10)
#anger:2
#fear:14
#happy:17
#love:19
#sadness:26
#surprise:27

emotions=['admiration','amusement','anger','annoyance','approval','caring','confusion','curiosity','desire','disappointment','disapproval','disgust','embarrassment','excitement'\
          ,'fear','gratitude','grief','happy','joy','love','nervousness','optimism','pride','realization','relief','remorse','sadness','surprise','neutral']

In [ ]:
for i in range(0, len(emotions)):
  df[emotions[i]]=0

In [ ]:
for i in range(df.shape[0]):
  result28=goemotions(df.loc[i,'Text'])
  idx=0
  for j in  result28[0]['labels']:
    df.loc[i, j]=result28[0]['scores'][idx]
    idx+=1


In [ ]:
df.head(50)

In [ ]:

for i in range(0, df.shape[0]):
  df.loc[i, 'Emotion']=emotions.index(df.loc[i, 'Emotion'])

In [ ]:
 df.to_csv("emotionsdf.csv")

이 위에까지는 실행하지 않으셔도 됩니다!!(시간이 오래걸림)

In [11]:
df=pd.read_csv('emotionsdf.csv')


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
X=df.drop(['Unnamed: 0','Unnamed: 0.1', 'Text', 'Emotion'], axis=1)
Y=df['Emotion']
x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.1, shuffle=True, stratify=Y, random_state=34)


In [45]:
from sklearn.decomposition import PCA
pca=PCA(n_components=10)
X_pca=pca.fit_transform(X)
x_train_pca, x_test_pca, y_train, y_test=train_test_split(X_pca, Y, test_size=0.1, shuffle=True, stratify=Y, random_state=34)


In [43]:
lr=LogisticRegression(C=10, max_iter=1000)
lr.fit(x_train, y_train)
pred=lr.predict(x_test)
accuracy_score(pred, y_test)

0.5820130475302889

In [46]:
lr.fit(x_train_pca, y_train)
pred=lr.predict(x_test_pca)
accuracy_score(pred, y_test)

0.5386766076421249

In [30]:
params={
    'max_depth':[10],
    'min_samples_split':[5,10,15,20]
}
grid_cv=GridSearchCV(dtree, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(x_train, y_train)
print('최고 정확도:', grid_cv.best_score_)
print('최적 하이퍼파라미터:', grid_cv.best_params_)

scores_df = pd.DataFrame(grid_cv.cv_results_)
scores_df[['rank_test_score', 'params', 'mean_test_score',  'split0_test_score',
           'split1_test_score', 'split2_test_score']]

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


최고 정확도: 0.5404649478671967
최적 하이퍼파라미터: {'max_depth': 10, 'min_samples_split': 20}


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.4s finished


,rank_test_score,params,mean_test_score,split0_test_score,split1_test_score,split2_test_score
0,2,"{'max_depth': 10, 'min_samples_split': 5}",0.537099,0.537123,0.534638,0.539537
1,4,"{'max_depth': 10, 'min_samples_split': 10}",0.533993,0.537745,0.529668,0.534566
2,3,"{'max_depth': 10, 'min_samples_split': 15}",0.534355,0.526561,0.543647,0.532857
3,1,"{'max_depth': 10, 'min_samples_split': 20}",0.540465,0.540385,0.541007,0.540003


In [ ]:
dtree=DecisionTreeClassifier(max_depth=15, max_features=20)
dtree.fit(x_train, y_train)
pred=dtree.predict(x_test)
accuracy_score(pred, y_test)

In [36]:
from sklearn.svm import LinearSVC, SVC

#svm = SVC(kernel='rbf', random_state=1, gamma=0.10, C=10.0)
svm=LinearSVC(C=0.1, loss="hinge")

svm.fit(x_train, y_train)
pred=svm.predict(x_test)
accuracy_score(pred, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5330848089468779

In [47]:
x_train

,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
13358,0.991514,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
11084,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.919497,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.34548,0.0,0.0,0.0
4807,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.974096,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.639746,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
7439,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.988168,0.0,0.00000,0.0,0.0,0.0
15694,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.950365,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12926,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.735360,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
17221,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.982235,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
4721,0.995292,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
12345,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.970655,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0


In [ ]:

my_text="Funeral ceremony...gloomy friday..."
goemotions(my_text)

[{'labels': ['excitement', 'joy'], 'scores': [0.7782779, 0.74246186]}]

In [ ]:
my_text=["Re-pinging @ghostridah14: why didn't you go to prom? BC my bf didn't like my friends",\
         "Funeral ceremony...gloomy friday...",\
         "@kelcouch I'm sorry at least it's Friday?",\
         "Wondering why I'm awake at 7am,writing a new song,plotting my evil secret plots muahahaha...oh damn ..."]


In [ ]:
my_df=pd.DataFrame()
for i in range(0, len(emotions)):
  my_df[emotions[i]]=0

In [ ]:
for i in range(len(my_text)):
  result28=goemotions(my_text[i])
  idx=0
  for j in  result28[0]['labels']:
    my_df.loc[i, j]=result28[0]['scores'][idx]
    idx+=1
my_df=my_df.fillna(0)
my_df.drop(['happy'], axis=1, inplace=True)

In [ ]:
for i in range(len(lr.predict(my_df))):
  print_emotion(lr.predict(my_df)[i])

'sadness'
'happy'
'sadness'
'surprise'


(21459, 28)